

---
# Project Setup

In [1]:
# Import libraries
import pandas as pd
import requests
import json
import boto3
from sqlalchemy import create_engine, text

In [2]:
# Load API keys and create key variable
openweather_apikey = pd.read_csv(r'C:\Users\rapha\My Drive\Work\jedha_dsfs\coursework\00_admin\api\openweather_apikey.csv')
openweather_apikey = openweather_apikey['api_key']

In [3]:
# Create list of cities
cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

# Create data frame where we will store our results
city_df = pd.DataFrame(columns = ['city', 'lat', 'lon'])
city_df['city'] = cities

# Query APIS to find best cities to travel to

### Query Nominatim API to get geolocalisation data

In [4]:
# Setup information for API request
url_coords = 'https://nominatim.openstreetmap.org/search'
headers_coords = {'User-Agent': 'Chrome/123.0.0.0'}

city_coords = city_df.copy()

In [5]:
# Iterate across cities, saving lattitude and longitude data from API
for i, city in enumerate(cities):
    payload_coords = {'city' : city,
                      'country' : "France",
                      'format' : 'json'}

    r_coords = requests.get(url_coords, params = payload_coords, headers = headers_coords)

    city_coords.iloc[i, 1] = r_coords.json()[0]['lat']
    city_coords.iloc[i, 2] = r_coords.json()[0]['lon']
    
city_coords.head(3)

,city,lat,lon
0,Mont Saint Michel,48.6359541,-1.511459954959514
1,St Malo,48.649518,-2.0260409
2,Bayeux,49.2764624,-0.7024738


In [6]:
# Set csv save path
path_city_coords = r'C:\Users\rapha\My Drive\Work\jedha_dsfs\coursework\p_kayak_data_collection\data\city_coords.csv'
# Save csv to system
city_coords.to_csv(path_city_coords, index = False)

### Query OpenWeather API to get weather data, using geolocalisation data

### Find best city to travel to, using weather data

In [7]:
# Setup information for API request
url_weather = 'https://api.openweathermap.org/data/2.5/forecast'

city_weather = city_coords.copy()

In [8]:
# Iterate across cities, saving forecast date, temperature, weather and chance of rain from API
for i, city in enumerate(cities):
    payload_weather = {'lat' : city_coords.iloc[i]['lat'],
                       'lon' : city_coords.iloc[i]['lon'],
                       'appid' : openweather_apikey}

    r_weather = requests.get(url_weather, params = payload_weather)

    city_weather.loc[i, 'dt'] = str(r_weather.json()['list'][-1]['dt'])
    city_weather.loc[i, 'feels_like'] = r_weather.json()['list'][-1]['main']['feels_like']
    city_weather.loc[i, 'weather_id'] = r_weather.json()['list'][-1]['weather'][0]['id']     # See https://openweathermap.org/weather-conditions for ID mapping
    city_weather.loc[i, 'pop'] = r_weather.json()['list'][-1]['pop']
    
city_weather.head(3)

,city,lat,lon,dt,feels_like,weather_id,pop
0,Mont Saint Michel,48.6359541,-1.511459954959514,1726390800,286.80,800.0,0.0
1,St Malo,48.649518,-2.0260409,1726390800,286.23,800.0,0.0
2,Bayeux,49.2764624,-0.7024738,1726390800,287.12,800.0,0.0


In [9]:
# Set csv save path
path_city_weather = r'C:\Users\rapha\My Drive\Work\jedha_dsfs\coursework\p_kayak_data_collection\data\city_weather.csv'
# Save csv to system
city_coords.to_csv(path_city_weather, index = False)

# Find best travel destinations based on weather

In [10]:
best_cities = city_weather.copy()

# Sort dataframe by chance of precipitation
best_cities.sort_values(by = 'pop')

# Assign global values to weather IDs
# Yes, this is very stupid, but I wanted to use more than just temp and PoP
def group_ids(ranges, value):
    return {i: value for r in ranges for i in range(r[0], r[1]+1)}

test_id_replacements = {
    **group_ids([(200, 232), (600, 622), (711, 711), (731, 781)], '3'),
    **group_ids([(300, 321), (500, 531)], '2'),
    **group_ids([(701, 701), (721, 721), (804, 804)], '1'),
    **group_ids([(800, 803)], '0')
}

best_cities['weather_id'] = best_cities['weather_id'].map(test_id_replacements).fillna(best_cities['weather_id'])

In [11]:
# Find difference from my optimal temperature
optimal_temperature = 25

best_cities['temp_dif'] = abs(best_cities['feels_like'] - (optimal_temperature + 273.15))

# sort cities by our weather attributes
best_cities = best_cities.sort_values(by = ['weather_id', 'pop', 'temp_dif'], ignore_index = True)

best_cities.head(3)

,city,lat,lon,dt,feels_like,weather_id,pop,temp_dif
0,Collioure,42.52505,3.0831554,1726390800,293.56,0,0.0,4.59
1,Bormes les Mimosas,43.1506968,6.3419285,1726390800,291.41,0,0.0,6.74
2,Cassis,43.2140359,5.5396318,1726390800,290.83,0,0.0,7.32


In [12]:
# Best 5 cities are the top 5 results
best_5_cities = best_cities.loc[0:4, 'city']
best_5_cities

0             Collioure
1    Bormes les Mimosas
2                Cassis
3                Ariege
4             Marseille
Name: city, dtype: object

In [13]:
# Set csv save path
path_best_5_cities = r'C:\Users\rapha\My Drive\Work\jedha_dsfs\coursework\p_kayak_data_collection\data\best_5_cities.csv'
# Save csv to system
best_5_cities.to_csv(path_best_5_cities, index = False)

# Run both booking.com crawlers

In [38]:
!python "C:\Users\rapha\My Drive\Work\jedha_dsfs\coursework\p_kayak_data_collection\booking_url_crawler_local.py"

2024-09-10 13:31:59 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2024-09-10 13:31:59 [scrapy.utils.log] INFO: Versions: lxml 5.2.2.0, libxml2 2.12.6, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.10.14 | packaged by Anaconda, Inc. | (main, Mar 21 2024, 16:20:14) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.0.14 4 Jun 2024), cryptography 42.0.5, Platform Windows-10-10.0.22631-SP0
2024-09-10 13:31:59 [scrapy.addons] INFO: Enabled addons:
[]
2024-09-10 13:31:59 [py.warnings] WARNING: c:\Users\rapha\anaconda3\envs\colab\lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.



In [39]:
!python "C:/Users/rapha/My Drive/Work/jedha_dsfs/coursework/p_kayak_data_collection/booking_data_crawler_local.py"

2024-09-10 13:32:03 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2024-09-10 13:32:03 [scrapy.utils.log] INFO: Versions: lxml 5.2.2.0, libxml2 2.12.6, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.10.14 | packaged by Anaconda, Inc. | (main, Mar 21 2024, 16:20:14) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.0.14 4 Jun 2024), cryptography 42.0.5, Platform Windows-10-10.0.22631-SP0
2024-09-10 13:32:03 [scrapy.addons] INFO: Enabled addons:
[]
2024-09-10 13:32:03 [py.warnings] WARNING: c:\Users\rapha\anaconda3\envs\colab\lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.



# Upload raw files to AWS S3

In [136]:
# Open data json
path_data = r'C:\Users\rapha\My Drive\Work\jedha_dsfs\coursework\p_kayak_data_collection\data\booking_data.json'
with open(path_data, 'r') as file:
    booking_data_raw = json.load(file)
    
path_data = r'C:\Users\rapha\My Drive\Work\jedha_dsfs\coursework\p_kayak_data_collection\data\booking_urls.json'
with open(path_data, 'r') as file:
    booking_urls_raw = json.load(file)

# Read key file
keys = pd.read_csv(r'C:\Users\rapha\My Drive\Work\jedha_dsfs\coursework\00_admin\aws\demojedhas3_accessKeys.csv')

# Start a session with credential access to S3
session = boto3.Session(aws_access_key_id=keys['Access key ID'][0],
                        aws_secret_access_key=keys['Secret access key'][0])

# Setup session resource and create a new bucket
s3 = session.resource("s3")
bucket = s3.create_bucket(Bucket="bucket-kayak-raw-10092024")

# Upload file to S3
bucket.put_object(Key='booking_data_raw.json', Body=json.dumps(booking_data_raw))
bucket.put_object(Key='booking_urls_raw.json', Body=json.dumps(booking_urls_raw))

s3.Object(bucket_name='bucket-kayak-raw-10092024', key='booking_urls_raw.json')

# Treat files to be uploaded to AWS RDS

In [137]:
raw_data_json = s3.Object('bucket-kayak-raw-10092024', 'booking_data_raw.json')
raw_data_json = raw_data_json.get()['Body'].read()

raw_urls_json = s3.Object('bucket-kayak-raw-10092024', 'booking_urls_raw.json')
raw_urls_json = raw_urls_json.get()['Body'].read()

raw_data_json = json.loads(raw_data_json)
raw_urls_json = json.loads(raw_urls_json)

data_data = pd.DataFrame.from_dict(raw_data_json)
urls_data = pd.DataFrame.from_dict(raw_urls_json)

In [143]:
clean_data = data_data.copy()

# Merge alt column onto base columns by null
clean_data['hotel_name'] = clean_data['hotel_name'].fillna(clean_data['hotel_name_alt'])
clean_data['address'] = clean_data['address'].fillna(clean_data['address_alt'])
clean_data['rating'] = clean_data['rating'].fillna(clean_data['rating_alt'])
clean_data['rating_amount'] = clean_data['rating_amount'].fillna(clean_data['rating_amount_alt'])

# Drop alternate columns and drop all rows with NA values
clean_data = clean_data.drop(['hotel_name_alt', 'address_alt', 'rating_alt', 'rating_amount_alt'], axis=1)
clean_data = clean_data.dropna()

# Convert rating column from X,x string to X.x float
clean_data['rating'] = clean_data['rating'].str.replace(',','.').astype('float64')

# Convert rating amount column to integer
clean_data['rating_amount'] = clean_data['rating_amount'].str.replace(' ','').str.extract('(\d+)').fillna('0').astype('int')

# Merge city names from URL dataframe
clean_data = clean_data.merge(urls_data, on='url')

clean_data.sample(3).head()

,url,hotel_name,address,rating,rating_amount,city
27,https://www.booking.com/hotel/fr/la-recampado-...,La Recampado,"\n27 Rue Rompi Cuou, 83230 Bormes-les-Mimosas,...",9.6,29,Bormes%20les%20Mimosas
25,https://www.booking.com/hotel/fr/village-vacan...,Village Vacances La Manne,"\n1329 Route de Cabasson, 83230 Bormes-les-Mim...",7.2,127,Bormes%20les%20Mimosas
40,https://www.booking.com/hotel/fr/chalet-bois-a...,Chalet bois au milieu des Pyrénées,"\n3 Las Planos, 09390 LʼHospitalet-près-lʼAndo...",8.9,180,Ariege


In [131]:
from sqlalchemy.engine import URL

url_object = URL.create(
    "postgresql+psycopg2",
    username="postgres",
    password="temp_postgre_pw",
    host="database-100924.czwimg606tju.us-east-1.rds.amazonaws.com",
    database="postgres",
)

# Create engine
engine = create_engine(url_object, echo=True, pool_pre_ping=True)
conn = engine.connect()

2024-09-10 19:15:24,650 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-09-10 19:15:24,651 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-09-10 19:15:24,901 INFO sqlalchemy.engine.Engine select current_schema()
2024-09-10 19:15:24,903 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-09-10 19:15:25,160 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-09-10 19:15:25,160 INFO sqlalchemy.engine.Engine [raw sql] {}


In [146]:
conn.execute(text('DROP TABLE IF EXISTS clean_booking_data'))
conn.commit()
clean_data.to_sql("clean_booking_data", engine)

2024-09-10 19:27:56,117 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS clean_booking_data
2024-09-10 19:27:56,118 INFO sqlalchemy.engine.Engine [cached since 11.28s ago] {}
2024-09-10 19:27:56,246 INFO sqlalchemy.engine.Engine COMMIT
2024-09-10 19:27:56,510 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-10 19:27:56,513 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2024-09-10 19:27:56,514 INFO sqlalchemy.engine.Engine [cached since 748.5s ago] {'table_name': 'clean_booking_data', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname

88

In [149]:
# Rollback any existing transaction
conn.rollback()

# Start a new transaction
conn.begin()

# Light SQL queries because why not
statement = text("SELECT * FROM clean_booking_data WHERE rating > 9")
result_high_rating = conn.execute(statement)
result_high_rating.fetchall()

statement = text("SELECT city, AVG(rating) AS average_rating FROM clean_booking_data GROUP BY city ORDER BY average_rating DESC")
result_reviews_city = conn.execute(statement)
result_reviews_city.fetchall()

2024-09-10 19:28:31,223 INFO sqlalchemy.engine.Engine ROLLBACK
2024-09-10 19:28:31,345 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-10 19:28:31,346 INFO sqlalchemy.engine.Engine SELECT * FROM clean_booking_data WHERE rating > 9
2024-09-10 19:28:31,348 INFO sqlalchemy.engine.Engine [cached since 678s ago] {}
2024-09-10 19:28:31,729 INFO sqlalchemy.engine.Engine SELECT city, AVG(rating) AS average_rating FROM clean_booking_data GROUP BY city ORDER BY average_rating DESC
2024-09-10 19:28:31,730 INFO sqlalchemy.engine.Engine [generated in 0.00106s] {}


[('Ariege', 9.1),
 ('Collioure', 8.6375),
 ('Bormes%20les%20Mimosas', 8.63684210526316),
 ('Cassis', 8.54736842105263),
 ('Marseille', 8.218750000000002)]